In [ ]:
# default_exp core

# Core

> This module contains all of the core information retrieval ranking metrics from: https://gist.github.com/bwhite/3726239.

In [1]:
# export
import numpy as np

In [ ]:
# export
def hit_rate_at_k(rs, k):
    """Score is percentage of first relevant item in list that occur
    at rank k or lower. First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: the largest rank position to consider
    Returns:
        Hit Rate @k
    """
    if k < 1 or k > len(rs[0]):
        raise ValueError('k value must be >=1 and < Max Rank')
    hits = 0
    for r in rs:
        if np.sum(r[:k]) > 0: hits += 1

    return hits / len(rs)

In [ ]:
HIT_1_VAL = 0.6666666666666666

relevancies = [[0, 1], [1, 1], [1, 0]]
h1 = hit_rate_at_k(relevancies, 1)

assert HIT_1_VAL == h1

HIT_2_VAL = 1.0

relevancies = [[0, 1], [1, 1], [1, 0]]
h2 = hit_rate_at_k(relevancies, 2)

assert HIT_2_VAL == h2

try:
    hit_rate_at_k(relevancies, -1)
    assert False
except Exception as e:
    assert type(e) == ValueError

try:
    hit_rate_at_k(relevancies, 4)
    assert False
except Exception as e:
    assert type(e) == ValueError

In [ ]:
# export
def mean_rank(rs):
    """Score is mean rank of the first relevant item in list
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean rank
    """
    _rs = []
    for r in rs:
        ids = np.asarray(r).nonzero()[0]
        if len(ids) == 0:
            _rs.append(0)
        else:
            _rs.append(ids[0] + 1)
    return np.mean(_rs)

In [ ]:
MR_VAL = 1.3333333333333333

relevancies = [[0, 1], [1, 1], [1, 0]]
mr = mean_rank(relevancies)

assert MR_VAL == mr

MR_VAL = 0.3333333333333333

relevancies = [[0, 1], [1, 1], [1, 0], [0, 0]]
mr = mean_rank(relevancies)

assert MR_VAL, mr

In [ ]:
# export
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [ ]:
MRR_VAL = 0.61111111111111105
relevancies = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
mrr = mean_reciprocal_rank(relevancies)

assert MRR_VAL == mrr

MRR_VAL = 0.5
relevancies = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
mrr = mean_reciprocal_rank(relevancies)

assert MRR_VAL == mrr

MRR_VAL = 0.75
relevancies = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
mrr = mean_reciprocal_rank(relevancies)

assert MRR_VAL == mrr

In [ ]:
# export
def r_precision(r):
    """Score is precision after all relevant documents have been retrieved
    Relevance is binary (nonzero is relevant).

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        R Precision
    """
    r = np.asarray(r) != 0
    z = r.nonzero()[0]
    if not z.size:
        return 0.
    return np.mean(r[:z[-1] + 1])

In [ ]:
PRECISION_VAL = 0.33333333333333331
relevancy = [0, 0, 1]
precision = r_precision(relevancy)

assert PRECISION_VAL == precision

PRECISION_VAL = 0.5
relevancy = [0, 1, 0]
precision = r_precision(relevancy)

assert PRECISION_VAL == precision

PRECISION_VAL = 1.0
relevancy = [1, 0, 0]
precision = r_precision(relevancy)

assert PRECISION_VAL == precision

In [8]:
# export
def r_recall(r, max_rel):
    """Score is recall after all relevant documents have been retrieved
    Relevance is binary (nonzero is relevant).

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        max_rel: Maximum number of documents that can be relevant
    Returns:
        R Recall
    """
    r = np.asarray(r) != 0
    z = r.nonzero()[0]
    if not z.size:
        return 0.
    if np.sum(r) > max_rel:
        raise ValueError('Number of relevant documents retrieved > max_rel')
    return np.sum(r) / max_rel

In [11]:
RECALL_VAL = 0.33333333333333331
relevancy = [0, 0, 1]
recall = r_recall(relevancy, 3)

assert RECALL_VAL == recall

RECALL_VAL = 0.5
relevancy = [0, 1, 0]
recall = r_recall(relevancy, 2)

assert RECALL_VAL == recall

RECALL_VAL = 0.
relevancy = [0, 0, 0]
recall = r_recall(relevancy, 3)

assert RECALL_VAL == recall

RECALL_VAL = 1.0
relevancy = [1, 0, 0]
recall = r_recall(relevancy, 1)

assert RECALL_VAL == recall

In [12]:
relevancy = [1, 1, 1]
try:
    r_recall(relevancy, 1)
    assert False
except Exception as e:
    assert type(e) == ValueError

In [ ]:
# export
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

In [ ]:
PRECISION_K_VAL = 0.0
relevancy = [0, 0, 1]
precision_k = precision_at_k(relevancy, 1)

assert PRECISION_K_VAL == precision_k

PRECISION_K_VAL = 0.0
precision_k = precision_at_k(relevancy, 2)

assert PRECISION_K_VAL == precision_k

PRECISION_K_VAL = 0.33333333333333331
precision_k = precision_at_k(relevancy, 3)

assert PRECISION_K_VAL == precision_k

In [ ]:
try:
    precision_at_k(relevancy, 4)
    assert False
except Exception as e:
    assert type(e) == ValueError

In [ ]:
# export
def average_precision(r):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

In [ ]:
relevancy = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
delta_r = 1. / sum(relevancy)
AVG_PRECISION_VAL = sum([sum(relevancy[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(relevancy) if y])

avg_precision = average_precision(relevancy)

assert AVG_PRECISION_VAL == avg_precision

In [ ]:
# export
def mean_average_precision(rs):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])


In [ ]:
MEAN_AVG_PRECISION_VAL = 0.78333333333333333

relevancies = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
mean_avg_precision = mean_average_precision(relevancies)

assert MEAN_AVG_PRECISION_VAL == mean_avg_precision

MEAN_AVG_PRECISION_VAL = 0.39166666666666666

relevancies = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
mean_avg_precision = mean_average_precision(relevancies)

assert MEAN_AVG_PRECISION_VAL == mean_avg_precision

In [ ]:
# export
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

In [ ]:
DCG_K_VAL = 3.0

relevancy = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
dcg_k = dcg_at_k(relevancy, 1)

assert DCG_K_VAL == dcg_k

DCG_K_VAL = 3.0

dcg_k = dcg_at_k(relevancy, 1, method=1)

assert DCG_K_VAL == dcg_k

DCG_K_VAL = 5.0

dcg_k = dcg_at_k(relevancy, 2)

assert DCG_K_VAL == dcg_k

DCG_K_VAL = 4.2618595071429155

dcg_k = dcg_at_k(relevancy, 2, method=1)

assert DCG_K_VAL == dcg_k

DCG_K_VAL = 9.6051177391888114

dcg_k = dcg_at_k(relevancy, 10)

assert DCG_K_VAL == dcg_k

DCG_K_VAL = 9.6051177391888114

dcg_k = dcg_at_k(relevancy, 11)

assert DCG_K_VAL == dcg_k

In [ ]:
# export
def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [ ]:
NDCG_K_VAL = 1.0

relevance = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
ndcg_k = ndcg_at_k(relevance, 1)

assert NDCG_K_VAL == ndcg_k

NDCG_K_VAL = 0.9203032077642922

relevance = [2, 1, 2, 0]
ndcg_k = ndcg_at_k(relevance, 4)

assert NDCG_K_VAL == ndcg_k

NDCG_K_VAL = 0.96519546960144276

ndcg_k = ndcg_at_k(relevance, 4, method=1)

assert NDCG_K_VAL == ndcg_k

NDCG_K_VAL = 0.0

ndcg_k = ndcg_at_k([0], 1)

assert NDCG_K_VAL == ndcg_k

NDCG_K_VAL = 1.0

ndcg_k = ndcg_at_k([1], 2)

assert NDCG_K_VAL == ndcg_k

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
